# Guess a Number - SageMaker notebook

Import the required modules

In [ ]:
import os
import sagemaker
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from sagemaker_common.misc import get_execution_role, wait_for_s3_object

**Job configuration**

In [ ]:
job_name_prefix = 'rl-guessnumber-gym'

local_mode = True

# Instance size configuration (if not running local)
if local_mode:
    instance_type = 'local'
else:
    instance_type = "ml.c5.xlarge"

**Find out AWS resources**

In [ ]:
# Figure out S3 bucket
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

# Figure out execution role
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

In [ ]:
# only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ./common/setup.sh

**RL Estimator** - here the training happens

In [ ]:
estimator = RLEstimator(entry_point="train-coach.py",
                        source_dir="sagemaker_job",
                        dependencies=["gym_guess_number"],
                        toolkit=RLToolkit.COACH,
                        toolkit_version='0.11.0',
                        framework=RLFramework.MXNET,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        hyperparameters = {
                          #"RLCOACH_PRESET": "preset-clippedppo",
                          #"rl.agent_params.algorithm.discount": 0.9,
                          #"rl.evaluation_steps:EnvironmentEpisodes": 8,
                          #"improve_steps": 10000,
                          "save_model": 0
                        }
                    )

estimator.fit(wait=True)